### 전처리


In [ ]:
train = pd.read_csv(r'C:\Users\USER\projects\0527p\train.csv', encoding='cp949')
import pandas as pd


## 변수들을 영문명으로 변경
cols = ['num', 'date_time', 'power', 'temp', 'wind','hum' ,'prec', 'sun', 'non_elec', 'solar']
train.columns = cols

# 시간 관련 변수들 생성
date = pd.to_datetime(train.date_time)
train['hour'] = date.dt.hour
train['day'] = date.dt.weekday
train['month'] = date.dt.month
train['week'] = date.dt.weekofyear


## 건물별, 요일별, 시간별 발전량 평균 넣어주기

power_mean = pd.pivot_table(train, values = 'power', index = ['num', 'hour', 'day'], aggfunc = np.mean).reset_index()
tqdm.pandas()
train['day_hour_mean'] = train.progress_apply(lambda x : power_mean.loc[(power_mean.num == x['num']) & (power_mean.hour == x['hour']) & (power_mean.day == x['day']) ,'power'].values[0], axis = 1)


## 건물별 시간별 발전량 평균 넣어주기
 
power_hour_mean = pd.pivot_table(train, values = 'power', index = ['num', 'hour'], aggfunc = np.mean).reset_index()
tqdm.pandas()
train['hour_mean'] = train.progress_apply(lambda x : power_hour_mean.loc[(power_hour_mean.num == x['num']) & (power_hour_mean.hour == x['hour']) ,'power'].values[0], axis = 1)


## 건물별 시간별 발전량 표준편차 넣어주기

power_hour_std = pd.pivot_table(train, values = 'power', index = ['num', 'hour'], aggfunc = np.std).reset_index()
tqdm.pandas()
train['hour_std'] = train.progress_apply(lambda x : power_hour_std.loc[(power_hour_std.num == x['num']) & (power_hour_std.hour == x['hour']) ,'power'].values[0], axis = 1)

### 공휴일 변수 추가
train['holiday'] = train.apply(lambda x : 0 if x['day']<5 else 1, axis = 1)
train.loc[('2020-08-17'<=train.date_time)&(train.date_time<'2020-08-18'), 'holiday'] = 1

## https://dacon.io/competitions/official/235680/codeshare/2366?page=1&dtype=recent
train['sin_time'] = np.sin(2*np.pi*train.hour/24)
train['cos_time'] = np.cos(2*np.pi*train.hour/24)

## https://dacon.io/competitions/official/235736/codeshare/2743?page=1&dtype=recent
train['THI'] = 9/5*train['temp'] - 0.55*(1-train['hum']/100)*(9/5*train['hum']-26)+32

def CDH(xs):
    ys = []
    for i in range(len(xs)):
        if i < 11:
            ys.append(np.sum(xs[:(i+1)]-26))
        else:
            ys.append(np.sum(xs[(i-11):(i+1)]-26))
    return np.array(ys)

cdhs = np.array([])
for num in range(1,61,1):
    temp = train[train['num'] == num]
    cdh = CDH(temp['temp'].values)
    cdhs = np.concatenate([cdhs, cdh])
train['CDH'] = cdhs

train.drop(['non_elec','solar','hour'], axis = 1, inplace = True)
train.head()

In [ ]:
## save the preprocessed data
train.to_csv(r'C:\Users\USER\projects\0527p\train_preprocessed.csv')

### ARIMA/SARIMA

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error

# CSV 파일에서 전체 데이터 불러오기
df = pd.read_csv(r'C:\Users\USER\projects\0527p\train_preprocessed.csv', parse_dates=['date_time'], index_col='date_time')

# 건물 번호 리스트 생성
building_nums = df['num'].unique()

# 각 건물에 대해 ARIMA 모델 적용 및 예측 시각화
for num in building_nums:
    # 건물별 데이터 추출
    building_data = df[df['num'] == num]['power']
    
    # 데이터 분할
    train_size = int(len(building_data) * 0.8)
    train, test = building_data.iloc[:train_size], building_data.iloc[train_size:]
    
    # ARIMA 모델 학습 및 예측
    model = ARIMA(train, order=(1, 1, 1))
    arima_fit = model.fit()
    pred = arima_fit.predict(start=len(train), end=len(train)+len(test)-1, typ='levels')
    
    # 평가 및 시각화
    mse = mean_squared_error(test, pred)
    print(f"Building {num} - Mean Squared Error: {mse}")
    
    plt.figure(figsize=(10, 5))
    plt.plot(train.index, train, label='Train')
    plt.plot(test.index, test, label='Test')
    plt.plot(test.index, pred, label='Predicted')
    plt.xlabel('Date')
    plt.ylabel('Power')
    plt.title(f'Building {num} - ARIMA Model Prediction')
    plt.legend()
    plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error

# CSV 파일에서 전체 데이터 불러오기
df = pd.read_csv(r'C:\Users\USER\projects\0527p\train_preprocessed.csv', parse_dates=['date_time'], index_col='date_time')

# 건물 번호 리스트 생성
building_nums = df['num'].unique()

# 각 건물에 대해 SARIMA 모델 적용 및 예측 시각화
for num in building_nums:
    # 건물별 데이터 추출
    building_data = df[df['num'] == num]['power']
    
    # 데이터 분할
    train_size = int(len(building_data) * 0.8)
    train, test = building_data.iloc[:train_size], building_data.iloc[train_size:]
    
    # SARIMA 모델 학습 및 예측
    model = SARIMAX(train, order=(1, 1, 1), seasonal_order=(1, 1, 1, 24))
    sarima_fit = model.fit(disp=False)
    pred = sarima_fit.predict(start=len(train), end=len(train)+len(test)-1, dynamic=False)
    
    # 평가 및 시각화
    mse = mean_squared_error(test, pred)
    print(f"Building {num} - Mean Squared Error: {mse}")
    
    plt.figure(figsize=(10, 5))
    plt.plot(train.index, train, label='Train')
    plt.plot(test.index, test, label='Test')
    plt.plot(test.index, pred, label='Predicted')
    plt.xlabel('Date')
    plt.ylabel('Power')
    plt.title(f'Building {num} - SARIMA Model Prediction')
    plt.legend()
    plt.show()


### ADF/PADF

In [ ]:

import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller

# 데이터 로드


train_df = pd.read_csv('train.csv',encoding='euc-kr', parse_dates=['date_time'])
test_df = pd.read_csv('test.csv', encoding='euc-kr', parse_dates=['date_time'])
sub = pd.read_csv('sample_submission.csv', encoding='euc-kr')
# renaming columns
train_df.columns = ['num','datetime','target','temperature','windspeed','humidity','precipitation','insolation','nelec_cool_flag','solar_flag']
test_df.columns = ['num','datetime','temperature','windspeed','humidity','precipitation','insolation','nelec_cool_flag','solar_flag']
data=train_df

# 'datetime' 열을 datetime 타입으로 변환
data['datetime'] = pd.to_datetime(data['datetime'])

# 'datetime'을 인덱스로 설정
data.set_index('datetime', inplace=True)

# 시계열 데이터를 시각화
plt.figure(figsize=(12, 6))
plt.plot(data['target'], label='Energy Usage (kWh)')
plt.title('Energy Usage Over Time')
plt.xlabel('Datetime')
plt.ylabel('Energy Usage (kWh)')
plt.legend()
plt.show()

# ADF 테스트 함수
def adf_test(series):
    result = adfuller(series)
    print('ADF Statistic:', result[0])
    print('p-value:', result[1])
    print('Critical Values:')
    for key, value in result[4].items():
        print(f'\t{key}: {value}')

# 'target' 열에 대해 ADF 테스트 수행
adf_test(data['target'])


In [ ]:
# 특정 빌딩 데이터 선택
building_num = 1
df = data[data['num'] == building_num]

# ACF 및 PACF 플롯 생성
plt.figure(figsize=(12, 6))

# ACF plot
plt.subplot(1, 2, 1)
plot_acf(df['target'], ax=plt.gca())
plt.title(f'ACF for Building {building_num}')

# PACF plot
plt.subplot(1, 2, 2)
plot_pacf(df['target'], method='ywm', ax=plt.gca())
plt.title(f'PACF for Building {building_num}')

plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
# ADF 테스트 함수
def adf_test(series):
    result = adfuller(series)
    print('ADF Statistic:', result[0])
    print('p-value:', result[1])
    print('Critical Values:')
    for key, value in result[4].items():
        print(f'\t{key}: {value}')

# 'target' 열에 대해 ADF 테스트 수행
adf_test(data['target'])

# ACF 및 PACF 플롯 생성
num_buildings = 60
fig, axes = plt.subplots(num_buildings, 2, figsize=(15, 3*num_buildings))

for num in range(1, num_buildings + 1):
    df = train_df[train_df.num == num]
    
    # ACF plot
    plot_acf(df['target'], ax=axes[num-1, 0])
    axes[num-1, 0].set_title(f'ACF for Building {num}')
    
    # PACF plot
    plot_pacf(df['target'], method='ywm', ax=axes[num-1, 1])
    axes[num-1, 1].set_title(f'PACF for Building {num}')

plt.tight_layout()
plt.show()

###  👌군집화

In [ ]:
import pandas as pd
train_df = pd.read_csv(r'C:\Users\USER\projects\0521p\train.csv', encoding='euc|-kr', parse_dates=['date_time'])
test_df = pd.read_csv(r'C:\Users\USER\projects\0521p\electricity_data\test.csv', encoding='euc-kr', parse_dates=['date_time'])
sub = pd.read_csv(r'C:\Users\USER\projects\0521p\sample_submission.csv', encoding='euc-kr')
# renaming columns
train_df.columns = ['num','datetime','target','temperature','windspeed','humidity','precipitation','insolation','nelec_cool_flag','solar_flag']
test_df.columns = ['num','datetime','temperature','windspeed','humidity','precipitation','insolation','nelec_cool_flag','solar_flag']

# 데이터 형태 및 개수, 결측치 확인
train_df.head(1)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns

# 날짜형식으로 바꿈
train_df['datetime'] = pd.to_datetime(train_df['datetime'])

# 시간 열 추가
train_df['hour'] = train_df['datetime'].dt.hour

hourly_mean = pd.DataFrame()

train_df['weekday'] = train_df['datetime'].dt.dayofweek
train_df['weekend'] = train_df['weekday'].apply(lambda x: 1 if x >= 5 else 0)

# 주중 데이터 필터링 (주중은 weekday가 0부터 4까지인 날짜)
weekday_data = train_df[train_df['weekday'].isin([0, 1, 2, 3, 4])]


In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

# 'datetime'을 datetime으로 변환하고, 시간 추출
train_df['datetime'] = pd.to_datetime(train_df['datetime'])
train_df['hour'] = train_df['datetime'].dt.hour
train_df['weekday'] = train_df['datetime'].dt.dayofweek

# 주중 데이터 필터링 (주중은 weekday가 0부터 4까지인 날짜)
weekday_data = train_df[train_df['weekday'].isin([0, 1, 2, 3, 4])]

# 각 건물 별, 시간대 별 평균 에너지 소비량 계산
hourly_consumption = weekday_data.groupby(['num', 'hour'])['target'].mean().unstack()

# 평균 에너지 소비량 데이터를 NumPy 배열로 변환
feature_matrix = hourly_consumption.fillna(0).values

# 엘보우 방법을 사용하여 최적의 클러스터 수 찾기
sse = []
silhouette_scores = []
k_range = range(2, 11)  # 2부터 10까지의 클러스터 수를 시도

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=0)
    kmeans.fit(feature_matrix)
    sse.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(feature_matrix, kmeans.labels_))

# # 엘보우 방법 그래프
# plt.figure(figsize=(10, 5))
# plt.plot(k_range, sse, marker='o')
# plt.title('Elbow Method For Optimal k')
# plt.xlabel('Number of clusters')
# plt.ylabel('Sum of squared distances (SSE)')
# plt.show()

# # 실루엣 계수 그래프
# plt.figure(figsize=(10, 5))
# plt.plot(k_range, silhouette_scores, marker='o')
# plt.title('Silhouette Scores For Optimal k')
# plt.xlabel('Number of clusters')
# plt.ylabel('Silhouette Score')
# plt.show()

# KMeans 군집화 수행 (최적의 k 선택 후, 예: 6)
optimal_k = 6  # 엘보우 방법과 실루엣 계수를 통해 결정된 최적의 클러스터 수
kmeans = KMeans(n_clusters=optimal_k, random_state=0)
clusters = kmeans.fit_predict(feature_matrix)

# 클러스터 결과를 보기 좋게 출력
clustered_buildings = {}
for building_idx, cluster_label in enumerate(clusters):
    if cluster_label not in clustered_buildings:
        clustered_buildings[cluster_label] = []
    clustered_buildings[cluster_label].append(building_idx + 1)  

# 클러스터 결과 출력
for cluster, buildings in clustered_buildings.items():
    print(f"클러스터 {cluster + 1}: 빌딩 {', '.join(map(str, buildings))}")

# # 시각화: 클러스터별 평균 에너지 소비 패턴
# plt.figure(figsize=(15, 10))

# for cluster_num in range(optimal_k):
#     cluster_data = hourly_consumption.iloc[np.where(clusters == cluster_num)].mean(axis=0)
    
#     plt.plot(cluster_data.index, cluster_data.values, label=f'Cluster {cluster_num + 1}')

# plt.title('Mean Hourly Energy Consumption by Cluster (Weekdays)')
# plt.xlabel('Hour of Day')
# plt.ylabel('Mean Energy Consumption')
# plt.legend(title='Cluster')
# plt.show()


In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# 데이터 변환 및 시간 추출
train_df['datetime'] = pd.to_datetime(train_df['datetime'])
train_df['hour'] = train_df['datetime'].dt.hour
train_df['weekday'] = train_df['datetime'].dt.dayofweek

# 주중 데이터 필터링 (주중은 weekday가 0부터 4까지인 날짜)
weekday_data = train_df[train_df['weekday'].isin([0, 1, 2, 3, 4])]
# 주말 데이터 필터링 (주말은 weekday가 5부터 6까지인 날짜)
weekend_data = train_df[train_df['weekday'].isin([5, 6])]

# 각 건물 별, 시간대 별 평균 에너지 소비량 계산 (주중)
weekday_hourly_consumption = weekday_data.groupby(['num', 'hour'])['target'].mean().unstack()
# 각 건물 별, 시간대 별 평균 에너지 소비량 계산 (주말)
weekend_hourly_consumption = weekend_data.groupby(['num', 'hour'])['target'].mean().unstack()

# 평균 에너지 소비량 데이터를 NumPy 배열로 변환 (주중, 주말)
weekday_feature_matrix = weekday_hourly_consumption.fillna(0).values
weekend_feature_matrix = weekend_hourly_consumption.fillna(0).values

# KMeans 군집화 수행 (주중: 3 클러스터, 주말: 4 클러스터)
optimal_k_weekday = 3
optimal_k_weekend = 4

kmeans_weekday = KMeans(n_clusters=optimal_k_weekday, random_state=0)
weekday_clusters = kmeans_weekday.fit_predict(weekday_feature_matrix)

kmeans_weekend = KMeans(n_clusters=optimal_k_weekend, random_state=0)
weekend_clusters = kmeans_weekend.fit_predict(weekend_feature_matrix)

def print_clusters(clusters, labels, title):
    clustered_buildings = {}
    for building_idx, cluster_label in enumerate(clusters):
        if labels[cluster_label] not in clustered_buildings:
            clustered_buildings[labels[cluster_label]] = []
        clustered_buildings[labels[cluster_label]].append(building_idx + 1)

    print(f"\n{title}")
    for cluster, buildings in clustered_buildings.items():
        print(f"클러스터 {cluster}: 빌딩 {', '.join(map(str, buildings))}")

# 주중 클러스터 결과 출력 (1, 2, 3)
weekday_labels = ['1', '2', '3']
print_clusters(weekday_clusters, weekday_labels, "Weekday Clusters")

# 주말 클러스터 결과 출력 (a, b, c, d)
weekend_labels = ['a', 'b', 'c', 'd']
print_clusters(weekend_clusters, weekend_labels, "Weekend Clusters")

# 클러스터 교차표 작성
cross_tab = pd.crosstab(
    pd.Series(weekday_clusters, name='Weekday Cluster').map({0: '1', 1: '2', 2: '3'}),
    pd.Series(weekend_clusters, name='Weekend Cluster').map({0: 'a', 1: 'b', 2: 'c', 3: 'd'})
)

# 보기 좋게 데이터프레임으로 출력
cross_tab_df = cross_tab.reset_index().rename_axis(None, axis=1)

# 스타일 적용하여 보기 좋게 출력
def style_crosstab(df):
    styled_df = df.style.set_table_styles(
        [{'selector': 'thead th', 'props': [('background-color', '#f7f7f9'), ('color', 'black')]}]
    ).set_properties(**{
        'background-color': '#e8e8e8', 
        'color': 'black', 
        'border-color': 'white',
        'text-align': 'center'
    }).set_caption('Cross Tabulation of Weekday and Weekend Clusters').format(na_rep='').hide(axis="index")
    return styled_df

styled_crosstab = style_crosstab(cross_tab_df)
display(styled_crosstab)

# # 시각화: 클러스터별 평균 에너지 소비 패턴 (주중, 주말)
# def plot_cluster_patterns(feature_matrix, clusters, optimal_k, labels, title):
#     plt.figure(figsize=(15, 10))
#     for cluster_num in range(optimal_k):
#         cluster_data = pd.DataFrame(feature_matrix).iloc[np.where(clusters == cluster_num)].mean(axis=0)
#         plt.plot(cluster_data.index, cluster_data.values, label=f'Cluster {labels[cluster_num]}')
#     plt.title(f'Mean Hourly Energy Consumption by Cluster ({title})')
#     plt.xlabel('Hour of Day')
#     plt.ylabel('Mean Energy Consumption')
#     plt.legend(title='Cluster')
#     # plt.show()

# plot_cluster_patterns(weekday_feature_matrix, weekday_clusters, optimal_k_weekday, weekday_labels, "Weekday")
# plot_cluster_patterns(weekend_feature_matrix, weekend_clusters, optimal_k_weekend, weekend_labels, "Weekend")


In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans

# 주중 클러스터 레이블과 주말 클러스터 레이블을 각각의 열로 추가
train_df['weekday_cluster'] = -1
train_df['weekend_cluster'] = -1

for idx, num in enumerate(train_df['num'].unique()):
    train_df.loc[train_df['num'] == num, 'weekday_cluster'] = weekday_clusters[idx]
    train_df.loc[train_df['num'] == num, 'weekend_cluster'] = weekend_clusters[idx]

# 각 클러스터 조합에 해당하는 건물 번호 출력
cluster_combinations = {}
weekday_labels = ['1', '2', '3']
weekend_labels = ['a', 'b', 'c', 'd']

for w_label in weekday_labels:
    for e_label in weekend_labels:
        combination_label = (w_label, e_label)
        cluster_combinations[combination_label] = []

# 각 건물의 클러스터 레이블을 확인하여 조합에 추가
unique_buildings = train_df['num'].unique()

for num in unique_buildings:
    weekday_cluster = weekday_clusters[num - 1]
    weekend_cluster = weekend_clusters[num - 1]
    
    w_label = weekday_labels[weekday_cluster]
    e_label = weekend_labels[weekend_cluster]
    
    combination_label = (w_label, e_label)
    cluster_combinations[combination_label].append(num)

# 각 클러스터 조합과 해당 건물 번호 출력
for combination, buildings in cluster_combinations.items():
    if buildings:
        print(f"클러스터 {combination}: 빌딩 {', '.join(map(str, buildings))}")
    else:
        print(f"클러스터 {combination}: 없음")


### 👌 프로펫 실행 코드

클러스터별로 변수를 생성하고, 해당되는 건물 번호만 모아서 저장한다.

In [ ]:
train_df.info()

In [ ]:
import pandas as pd

# 각 클러스터 조합에 해당하는 빌딩 번호
cluster_combinations = {
    ('1', 'a'): [3, 5, 15, 24, 26, 28, 32, 33, 42, 44, 52, 57, 60],
    ('1', 'b'): [],
    ('1', 'c'): [18, 37, 46, 47, 55],
    ('1', 'd'): [11, 40],
    ('2', 'a'): [],
    ('2', 'b'): [1, 8, 30, 31, 38, 54],
    ('2', 'c'): [],
    ('2', 'd'): [],
    ('3', 'a'): [],
    ('3', 'b'): [],
    ('3', 'c'): [2, 4, 6, 7, 9, 13, 14, 16, 17, 19, 20, 21, 22, 23, 25, 27, 29, 34, 35, 36, 39, 41, 43, 45, 48, 49, 50, 51, 53, 56, 58, 59],
    ('3', 'd'): [10, 12]
}

# 클러스터 조합별 데이터셋 저장할 딕셔너리
clustered_dfs = {}

# 각 클러스터 조합에 대해 데이터프레임 필터링 및 저장
for combination, buildings in cluster_combinations.items():
    if buildings:
        clustered_dfs[combination] = train_df[train_df['num'].isin(buildings)]
    else:
        clustered_dfs[combination] = pd.DataFrame(columns=train_df.columns)

# 클러스터별 데이터프레임을 변수에 할당
for combination, df in clustered_dfs.items():
    var_name = f"cluster_{combination[0]}_{combination[1]}"
    globals()[var_name] = df

# 확인을 위해 각 클러스터 조합별 데이터프레임의 크기 출력
for combination in cluster_combinations.keys():
    var_name = f"cluster_{combination[0]}_{combination[1]}"
    print(f"{var_name}: {len(globals()[var_name])} rows")


위 결과처럼 12개의 클러스터로 나누었다.

In [ ]:
cluster_1_a.info()

df = cluster_1_a에 대하여 프로펫을 진행하면 이래와 같다.

#### 😎 군집별로 각각 프로펫 돌려본 결과

In [ ]:
pip install prophet


In [ ]:
import pandas as pd
from prophet import Prophet
import matplotlib.pyplot as plt

# 클러스터 데이터셋 딕셔너리
clusters = {
    'cluster_1_a': cluster_1_a,
    # 'cluster_1_b': cluster_1_b,
    'cluster_1_c': cluster_1_c,
    'cluster_1_d': cluster_1_d,
    # 'cluster_2_a': cluster_2_a,
    'cluster_2_b': cluster_2_b,
    # 'cluster_2_c': cluster_2_c,
    # 'cluster_2_d': cluster_2_d,
    # 'cluster_3_a': cluster_3_a,
    # 'cluster_3_b': cluster_3_b,
    'cluster_3_c': cluster_3_c,
    'cluster_3_d': cluster_3_d
}

# 클러스터별로 Prophet 모델 학습 및 예측
for cluster_name, df in clusters.items():
    if df.empty:
        print(f"{cluster_name}: No data available")
        continue
    
    # Prophet 입력 형식에 맞게 데이터프레임 변환
    df = df.rename(columns={'datetime': 'ds', 'target': 'y'})
    
    # Prophet 모델 생성 및 학습
    model = Prophet()
    model.fit(df)
    
    # 미래 데이터프레임 생성 (예측할 기간 설정, 예: 20일)
    future = model.make_future_dataframe(periods=20)
    
    # 예측
    forecast = model.predict(future)
    
    # 예측 결과 시각화
    fig1 = model.plot(forecast)
    plt.title(f'{cluster_name} - Electricity Consumption Forecast')
    plt.xlabel('Date')
    plt.ylabel('Electricity Consumption (target)')
    
    # 검은색 점 크기 줄이기
    for line in fig1.gca().get_lines():
        if line.get_marker() == 'o':
            line.set_markersize(1)  # 점의 크기를 1로 설정
    
    plt.show()

#### 😎 결과 해석

예측 그래프 (`fig1`)

1. **검은색 점 (실제 데이터 포인트)**:
   - 이 점들은 실제 전력 소비 데이터를 나타냅니다. `cluster_1_a` 데이터셋의 `target` 값으로, 시간(`datetime`)에 따른 실제 전력 소비량을 보여줍니다.

2. **파란색 선 (예측 값)**:
   - 이 선은 Prophet 모델이 예측한 전력 소비량을 나타냅니다. 모델이 학습한 후 예측한 값으로, 실제 데이터와 비교하여 모델의 예측 성능을 평가할 수 있습니다.

3. **파란색 음영 영역 (불확실성 범위)**:
   - 이 영역은 예측의 불확실성 범위를 나타냅니다. Prophet 모델이 예측한 값의 신뢰 구간을 나타내며, 예측이 얼마나 정확한지를 나타냅니다.

4. **날짜 축 (X축)**:
   - X축은 날짜를 나타냅니다. 예측이 이루어진 시간 범위를 보여줍니다.

5. **전력 소비량 축 (Y축)**:
   - Y축은 전력 소비량을 나타냅니다. `target` 값으로, 예측된 전력 소비량의 단위를 나타냅니다.

6. **제목 및 축 라벨**:
   - 그래프의 제목은 "Electricity Consumption Forecast"로, 전력 소비 예측임을 나타냅니다. X축 라벨은 "Date", Y축 라벨은 "Electricity Consumption (target)"입니다.

컴포넌트 그래프 (`fig2`)

컴포넌트 그래프는 Prophet 모델의 예측 결과를 더 자세히 이해할 수 있도록 도와줍니다. 이 그래프는 예측에 기여하는 주요 요소들을 보여줍니다.

1. **추세 (Trend)**:
   - 전반적인 추세를 나타내며, 시간에 따른 전력 소비의 장기적인 변화 패턴을 보여줍니다.

2. **주기성 (Yearly, Weekly, Daily Seasonality)**:
   - 특정 주기(연간, 주간, 일간)마다 반복되는 패턴을 나타냅니다. 예를 들어, 전력 소비가 특정 시간대나 요일에 반복되는 패턴을 보여줍니다.

3. **기타 요소 (Holidays, Special Events)**:
   - 공휴일이나 특별한 이벤트와 같은 특정 이벤트에 따른 변화를 나타냅니다. 여기서는 특별히 추가된 이벤트가 없다면 표시되지 않을 수 있습니다.

결론

이 예측 결과를 통해 전력 소비 패턴을 이해하고, 미래의 전력 수요를 예측할 수 있습니다. 실제 데이터 포인트(검은색 점)와 모델 예측 값(파란색 선)을 비교하여 모델의 성능을 평가할 수 있으며, 불확실성 범위(파란색 음영)를 통해 예측의 신뢰도를 확인할 수 있습니다.

####  😎 1~60번 건물을 전부 각각 프로펫 돌려본 결과

In [ ]:
# 고유한 num 값을 추출
unique_nums = train_df['num'].unique()

# 각 num 값에 대해 데이터프레임을 필터링하고 변수에 할당
for num in unique_nums:
    globals()[f'num_{num}'] = train_df[train_df['num'] == num]


In [ ]:
import pandas as pd
from prophet import Prophet
import matplotlib.pyplot as plt

# 클러스터 데이터셋 딕셔너리 생성
clusters = {f'num_{i}': globals()[f'num_{i}'] for i in range(1, 61)}

# 클러스터별로 Prophet 모델 학습 및 예측
for cluster_name, df in clusters.items():
    if df.empty:
        print(f"{cluster_name}: No data available")
        continue
    
    # Prophet 입력 형식에 맞게 데이터프레임 변환
    df = df.rename(columns={'datetime': 'ds', 'target': 'y'})
    
    # Prophet 모델 생성 및 학습
    model = Prophet()
    model.fit(df)
    
    # 미래 데이터프레임 생성 (예측할 기간 설정, 예: 20일)
    future = model.make_future_dataframe(periods=20)
    
    # 예측
    forecast = model.predict(future)
    
    # 예측 결과 시각화
    fig1 = model.plot(forecast)
    plt.title(f'{cluster_name} - Electricity Consumption Forecast')
    plt.xlabel('Date')
    plt.ylabel('Electricity Consumption (target)')
    
    # 검은색 점 크기 줄이기
    for line in fig1.gca().get_lines():
        if line.get_marker() == 'o':
            line.set_markersize(1)  # 점의 크기를 1로 설정
    
    plt.show()


아래는 Prophet 모델의 학습 결과를 RMSE값을 구하여 비교했다. RMSE값이 가장 낮은 건물은 32번 (18.9)이고, 가장 높은 건물은 8번(1945.2)이다.

In [ ]:
# 성능 지표를 저장할 리스트
performance_metrics = []

# 클러스터별로 Prophet 모델 학습 및 예측
for cluster_name, df in clusters.items():
    if df.empty:
        print(f"{cluster_name}: No data available")
        continue
    
    # Prophet 입력 형식에 맞게 데이터프레임 변환
    df = df.rename(columns={'datetime': 'ds', 'target': 'y'})
    
    # Prophet 모델 생성 및 학습
    model = Prophet()
    model.fit(df)
    
    # 미래 데이터프레임 생성 (예측할 기간 설정, 예: 20일)
    future = model.make_future_dataframe(periods=20)
    
    # 예측
    forecast = model.predict(future)
    
    # 성능 지표 계산 (예: MAE, RMSE)
    y_true = df['y'].values
    y_pred = forecast['yhat'][:len(y_true)].values  # 예측 결과 중 실제 데이터 길이만큼 사용
    
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    
    performance_metrics.append({
        'Building': cluster_name,
        'MAE': mae,
        'RMSE': rmse
    })

# 성능 지표를 데이터프레임으로 변환
performance_df = pd.DataFrame(performance_metrics)

# RMSE 값으로 데이터프레임 정렬
performance_df = performance_df.sort_values(by='RMSE')

# 성능 지표 출력
print(performance_df)

# 성능 지표 시각화 (RMSE만)
performance_df.plot(x='Building', y='RMSE', kind='bar', figsize=(15, 7), legend=False)
plt.title('Prediction RMSE by Building (Sorted)')
plt.xlabel('Building')
plt.ylabel('RMSE')
plt.xticks(rotation=90)
plt.show()


### 👌 LSTM 돌려보기

#### 🐚 1~60번 건물 모두 돌려보기

In [ ]:
# 고유한 num 값을 추출
unique_nums = train_df['num'].unique()

# 각 num 값에 대해 데이터프레임을 필터링하고 변수에 할당
for num in unique_nums:
    globals()[f'num_{num}'] = train_df[train_df['num'] == num]


In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
import matplotlib.pyplot as plt

# 클러스터 데이터셋 로드
df = num_1.copy()

# 데이터프레임을 'datetime'을 인덱스로 설정
df.set_index('datetime', inplace=True)

# 데이터 스케일링
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df['target'].values.reshape(-1, 1))

# LSTM 입력 형식에 맞게 데이터 준비
sequence_length = 20
x_train, y_train = [], []

for i in range(sequence_length, len(scaled_data)):
    x_train.append(scaled_data[i-sequence_length:i, 0])
    y_train.append(scaled_data[i, 0])

x_train, y_train = np.array(x_train), np.array(y_train)

# LSTM 입력 형식에 맞게 데이터 재구성
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

# LSTM 모델 생성
model = Sequential()
model.add(LSTM(units=100, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(LSTM(units=100))
model.add(Dense(units=1))

# 모델 컴파일 및 학습
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_train, y_train, epochs=50, batch_size=32)

# 미래 데이터를 예측하기 위해 준비
test_data = scaled_data[-sequence_length:]
x_test = [test_data]

x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

# 예측
predicted_values = []

for _ in range(200):
    prediction = model.predict(x_test)
    predicted_values.append(prediction[0])
    new_test_data = np.append(x_test[0][1:], prediction[0])
    x_test = np.reshape(new_test_data, (1, sequence_length, 1))

predicted_values = scaler.inverse_transform(predicted_values)

# 예측 결과 시각화
plt.figure(figsize=(14, 5))
plt.plot(df.index, df['target'], label='Actual')
plt.plot(pd.date_range(start=df.index[-1], periods=201, freq='H')[1:], predicted_values, label='Predicted', color='red')
plt.title('1A Electricity Consumption Forecast')
plt.xlabel('Date')
plt.ylabel('Electricity Consumption (target)')
plt.legend()
plt.show()

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
import matplotlib.pyplot as plt

# 클러스터 데이터셋 로드
df = num_60.copy()

# 데이터프레임을 'datetime'을 인덱스로 설정
df.set_index('datetime', inplace=True)

# 데이터 스케일링
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df['target'].values.reshape(-1, 1))

# LSTM 입력 형식에 맞게 데이터 준비
sequence_length = 20
x_train, y_train = [], []

for i in range(sequence_length, len(scaled_data)):
    x_train.append(scaled_data[i-sequence_length:i, 0])
    y_train.append(scaled_data[i, 0])

x_train, y_train = np.array(x_train), np.array(y_train)

# LSTM 입력 형식에 맞게 데이터 재구성
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

# LSTM 모델 생성
model = Sequential()
model.add(LSTM(units=100, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(LSTM(units=100))
model.add(Dense(units=1))

# 모델 컴파일 및 학습
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_train, y_train, epochs=50, batch_size=32)

# 미래 데이터를 예측하기 위해 준비
test_data = scaled_data[-sequence_length:]
x_test = [test_data]

x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

# 예측
predicted_values = []

for _ in range(200):
    prediction = model.predict(x_test)
    predicted_values.append(prediction[0])
    new_test_data = np.append(x_test[0][1:], prediction[0])
    x_test = np.reshape(new_test_data, (1, sequence_length, 1))

predicted_values = scaler.inverse_transform(predicted_values)

# 예측 결과 시각화
plt.figure(figsize=(14, 5))
plt.plot(df.index, df['target'], label='Actual')
plt.plot(pd.date_range(start=df.index[-1], periods=201, freq='H')[1:], predicted_values, label='Predicted', color='red')
plt.title('Electricity Consumption Forecast')
plt.xlabel('Date')
plt.ylabel('Electricity Consumption (target)')
plt.legend()
plt.show()

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
import matplotlib.pyplot as plt

# 클러스터 데이터셋 로드
clusters = {f'num_{i}': globals()[f'num_{i}'].copy() for i in range(1, 61)}

# LSTM 모델 정의 함수
def create_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(units=100, return_sequences=True, input_shape=input_shape))
    model.add(LSTM(units=100))
    model.add(Dense(units=1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# 학습 및 예측 함수 정의
def train_and_predict(df, sequence_length=20, epochs=50, batch_size=32, future_steps=200):
    # 데이터프레임을 'datetime'을 인덱스로 설정
    df.set_index('datetime', inplace=True)
    
    # 데이터 스케일링
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(df['target'].values.reshape(-1, 1))

    # LSTM 입력 형식에 맞게 데이터 준비
    x_train, y_train = [], []
    for i in range(sequence_length, len(scaled_data)):
        x_train.append(scaled_data[i-sequence_length:i, 0])
        y_train.append(scaled_data[i, 0])

    x_train, y_train = np.array(x_train), np.array(y_train)
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

    # LSTM 모델 생성 및 학습
    model = create_lstm_model((x_train.shape[1], 1))
    model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size)

    # 미래 데이터를 예측하기 위해 준비
    test_data = scaled_data[-sequence_length:]
    x_test = [test_data]
    x_test = np.array(x_test)
    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

    # 예측
    predicted_values = []
    for _ in range(future_steps):
        prediction = model.predict(x_test)
        predicted_values.append(prediction[0])
        new_test_data = np.append(x_test[0][1:], prediction[0])
        x_test = np.reshape(new_test_data, (1, sequence_length, 1))

    predicted_values = scaler.inverse_transform(predicted_values)
    
    # 예측 결과 시각화
    plt.figure(figsize=(14, 5))
    plt.plot(df.index, df['target'], label='Actual')
    plt.plot(pd.date_range(start=df.index[-1], periods=future_steps + 1, freq='H')[1:], predicted_values, label='Predicted', color='red')
    plt.title(f'{df.name} Electricity Consumption Forecast')
    plt.xlabel('Date')
    plt.ylabel('Electricity Consumption (target)')
    plt.legend()
    plt.show()

# 모든 클러스터에 대해 모델 학습 및 예측
for cluster_name, cluster_df in clusters.items():
    cluster_df.name = cluster_name
    print(f'Processing {cluster_name}...')
    train_and_predict(cluster_df)


In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

# LSTM 모델 정의 함수
def create_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(units=100, return_sequences=True, input_shape=input_shape))
    model.add(LSTM(units=100))
    model.add(Dense(units=1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# 학습 및 예측 함수 정의
def train_and_evaluate(df, sequence_length=20, epochs=50, batch_size=32):
    # 데이터프레임을 'datetime'을 인덱스로 설정
    df.set_index('datetime', inplace=True)
    
    # 데이터 스케일링
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(df['target'].values.reshape(-1, 1))

    # LSTM 입력 형식에 맞게 데이터 준비
    x_train, y_train = [], []
    for i in range(sequence_length, int(len(scaled_data) * 0.8)):
        x_train.append(scaled_data[i-sequence_length:i, 0])
        y_train.append(scaled_data[i, 0])

    x_train, y_train = np.array(x_train), np.array(y_train)
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

    # LSTM 모델 생성 및 학습
    model = create_lstm_model((x_train.shape[1], 1))
    model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0)

    # 테스트 데이터 준비
    test_data = scaled_data[int(len(scaled_data) * 0.8) - sequence_length:]
    x_test, y_test = [], []
    for i in range(sequence_length, len(test_data)):
        x_test.append(test_data[i-sequence_length:i, 0])
        y_test.append(test_data[i, 0])

    x_test, y_test = np.array(x_test), np.array(y_test)
    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

    # 예측
    predictions = model.predict(x_test)
    predictions = scaler.inverse_transform(predictions)
    y_test = scaler.inverse_transform(y_test.reshape(-1, 1))

    # RMSE 계산
    rmse = np.sqrt(mean_squared_error(y_test, predictions))
    
    return rmse

# 모든 클러스터에 대해 RMSE 계산
rmses = {}
for i in range(1, 61):  # 예제에서는 num_1부터 num_3까지 처리합니다. 실제로는 (1, 61)로 변경하세요.
    cluster_name = f'num_{i}'
    print(f'Processing {cluster_name}...')
    df = globals()[cluster_name].copy()
    rmse = train_and_evaluate(df)
    rmses[cluster_name] = rmse
    print(f'Building {cluster_name} - RMSE: {rmse}')

# RMSE 값 오름차순으로 정렬
sorted_rmses = dict(sorted(rmses.items(), key=lambda item: item[1]))

# RMSE 값을 데이터프레임으로 정리하여 출력
rmse_df = pd.DataFrame(list(sorted_rmses.items()), columns=['Cluster', 'RMSE'])
print(rmse_df)

# RMSE 결과 시각화
plt.figure(figsize=(14, 7))
plt.bar(rmse_df['Cluster'], rmse_df['RMSE'])
plt.xlabel('Cluster')
plt.ylabel('RMSE')
plt.title('RMSE of LSTM Model for Each Cluster')
plt.xticks(rotation=90)
plt.show()


In [ ]:
# 단변량으로 돌렸을 때
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
import matplotlib.pyplot as plt

# LSTM 모델 정의 함수
def create_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(units=100, return_sequences=True, input_shape=input_shape))
    model.add(LSTM(units=100))
    model.add(Dense(units=1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# SMAPE 계산 함수 정의
def smape(y_true, y_pred):
    return 100/len(y_true) * np.sum(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))

# 학습 및 예측 함수 정의
def train_and_evaluate(df, sequence_length=20, epochs=50, batch_size=32):
    # 데이터프레임을 'datetime'을 인덱스로 설정
    df.set_index('datetime', inplace=True)
    
    # 데이터 스케일링
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(df['target'].values.reshape(-1, 1))

    # LSTM 입력 형식에 맞게 데이터 준비
    x_train, y_train = [], []
    for i in range(sequence_length, int(len(scaled_data) * 0.8)):
        x_train.append(scaled_data[i-sequence_length:i, 0])
        y_train.append(scaled_data[i, 0])

    x_train, y_train = np.array(x_train), np.array(y_train)
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

    # LSTM 모델 생성 및 학습
    model = create_lstm_model((x_train.shape[1], 1))
    model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0)

    # 테스트 데이터 준비
    test_data = scaled_data[int(len(scaled_data) * 0.8) - sequence_length:]
    x_test, y_test = [], []
    for i in range(sequence_length, len(test_data)):
        x_test.append(test_data[i-sequence_length:i, 0])
        y_test.append(test_data[i, 0])

    x_test, y_test = np.array(x_test), np.array(y_test)
    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

    # 예측
    predictions = model.predict(x_test)
    predictions = scaler.inverse_transform(predictions)
    y_test = scaler.inverse_transform(y_test.reshape(-1, 1))

    # SMAPE 계산
    smape_value = smape(y_test, predictions)
    
    return smape_value

# 모든 클러스터에 대해 SMAPE 계산
smapes = {}
for i in range(1, 61):  # 예제에서는 num_1부터 num_60까지 처리합니다.
    cluster_name = f'num_{i}'
    print(f'Processing {cluster_name}...')
    df = globals()[cluster_name].copy()
    smape_value = train_and_evaluate(df)
    smapes[cluster_name] = smape_value
    print(f'Building {cluster_name} - SMAPE: {smape_value}')

# SMAPE 값 오름차순으로 정렬
sorted_smapes = dict(sorted(smapes.items(), key=lambda item: item[1]))

# SMAPE 값을 데이터프레임으로 정리하여 출력
smape_df = pd.DataFrame(list(sorted_smapes.items()), columns=['Cluster', 'SMAPE'])
print(smape_df)

# SMAPE 결과 시각화
plt.figure(figsize=(14, 7))
plt.bar(smape_df['Cluster'], smape_df['SMAPE'])
plt.xlabel('Cluster')
plt.ylabel('SMAPE')
plt.title('SMAPE of LSTM Model for Each Cluster')
plt.xticks(rotation=90)
plt.show()


In [ ]:
# 다변량으로 돌렸을 때
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
import matplotlib.pyplot as plt

# LSTM 모델 정의 함수
def create_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(units=100, return_sequences=True, input_shape=input_shape))
    model.add(LSTM(units=100))
    model.add(Dense(units=1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# SMAPE 계산 함수 정의
def smape(y_true, y_pred):
    return 100 / len(y_true) * np.sum(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))

# 학습 및 예측 함수 정의
def train_and_evaluate(df, sequence_length=20, epochs=50, batch_size=32):
    # 데이터프레임을 'datetime'을 인덱스로 설정
    df.set_index('datetime', inplace=True)
    
    # 독립변수와 종속변수 분리
    features = df.drop(columns=['target'])
    target = df['target']

    # 데이터 스케일링
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_features = scaler.fit_transform(features)
    scaled_target = scaler.fit_transform(target.values.reshape(-1, 1))

    # LSTM 입력 형식에 맞게 데이터 준비
    x_train, y_train = [], []
    for i in range(sequence_length, int(len(scaled_target) * 0.8)):
        x_train.append(scaled_features[i-sequence_length:i, :])
        y_train.append(scaled_target[i, 0])

    x_train, y_train = np.array(x_train), np.array(y_train)
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], x_train.shape[2]))

    # LSTM 모델 생성 및 학습
    model = create_lstm_model((x_train.shape[1], x_train.shape[2]))
    model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0)

    # 테스트 데이터 준비
    test_data_features = scaled_features[int(len(scaled_target) * 0.8) - sequence_length:]
    test_data_target = scaled_target[int(len(scaled_target) * 0.8) - sequence_length:]
    
    x_test, y_test = [], []
    for i in range(sequence_length, len(test_data_target)):
        x_test.append(test_data_features[i-sequence_length:i, :])
        y_test.append(test_data_target[i, 0])

    x_test, y_test = np.array(x_test), np.array(y_test)
    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], x_test.shape[2]))

    # 예측
    predictions = model.predict(x_test)
    predictions = scaler.inverse_transform(np.column_stack([np.zeros((predictions.shape[0], scaled_features.shape[1]-1)), predictions]))[:, -1]
    y_test = scaler.inverse_transform(np.column_stack([np.zeros((y_test.shape[0], scaled_features.shape[1]-1)), y_test.reshape(-1, 1)]))[:, -1]

    # SMAPE 계산
    smape_value = smape(y_test, predictions)
    
    return smape_value

# 모든 클러스터에 대해 SMAPE 계산
smapes = {}
for i in range(1, 61):  # 예제에서는 num_1부터 num_60까지 처리합니다.
    cluster_name = f'num_{i}'
    print(f'Processing {cluster_name}...')
    df = globals()[cluster_name].copy()
    smape_value = train_and_evaluate(df)
    smapes[cluster_name] = smape_value
    print(f'Building {cluster_name} - SMAPE: {smape_value}')

# SMAPE 값 오름차순으로 정렬
sorted_smapes = dict(sorted(smapes.items(), key=lambda item: item[1]))

# SMAPE 값을 데이터프레임으로 정리하여 출력
smape_df = pd.DataFrame(list(sorted_smapes.items()), columns=['Cluster', 'SMAPE'])
print(smape_df)

# SMAPE 결과 시각화
plt.figure(figsize=(14, 7))
plt.bar(smape_df['Cluster'], smape_df['SMAPE'])
plt.xlabel('Cluster')
plt.ylabel('SMAPE')
plt.title('SMAPE of LSTM Model for Each Cluster')
plt.xticks(rotation=90)
plt.show()


In [ ]:
num_1.info()

###  👌클러스터별로 아리마 돌려본 결과

In [ ]:
cluster_1_a.info()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


# 'datetime' 컬럼을 datetime 형식으로 변환
cluster_1_a['datetime'] = pd.to_datetime(cluster_1_a['datetime'])

# unique 'num' 값들
unique_nums = cluster_1_a['num'].unique()

# Plotting
plt.figure(figsize=(15, 10))

for num in unique_nums:
    subset = cluster_1_a[cluster_1_a['num'] == num]
    plt.plot(subset['datetime'], subset['target'], label=f'num {num}')

plt.xlabel('Datetime')
plt.ylabel('Target')
plt.title('Target over Time for Each num')
plt.legend()
plt.show()


In [ ]:
# 'datetime' 컬럼을 datetime 형식으로 변환
cluster_1_c['datetime'] = pd.to_datetime(cluster_1_c['datetime'])

# unique 'num' 값들
unique_nums = cluster_1_c['num'].unique()

# Plotting
plt.figure(figsize=(15, 10))

for num in unique_nums:
    subset = cluster_1_c[cluster_1_c['num'] == num]
    plt.plot(subset['datetime'], subset['target'], label=f'num {num}')

plt.xlabel('Datetime')
plt.ylabel('Target')
plt.title('Target over Time for Each num')
plt.legend()
plt.show()

In [ ]:
# 'datetime' 컬럼을 datetime 형식으로 변환
cluster_1_d['datetime'] = pd.to_datetime(cluster_1_d['datetime'])

# unique 'num' 값들
unique_nums = cluster_1_d['num'].unique()

# Plotting
plt.figure(figsize=(15, 10))

for num in unique_nums:
    subset = cluster_1_d[cluster_1_d['num'] == num]
    plt.plot(subset['datetime'], subset['target'], label=f'num {num}')

plt.xlabel('Datetime')
plt.ylabel('Target')
plt.title('Target over Time for Each num')
plt.legend()
plt.show()

In [ ]:
# 'datetime' 컬럼을 datetime 형식으로 변환
cluster_2_b['datetime'] = pd.to_datetime(cluster_2_b['datetime'])

# unique 'num' 값들
unique_nums = cluster_2_b['num'].unique()

# Plotting
plt.figure(figsize=(15, 10))

for num in unique_nums:
    subset = cluster_2_b[cluster_2_b['num'] == num]
    plt.plot(subset['datetime'], subset['target'], label=f'num {num}')

plt.xlabel('Datetime')
plt.ylabel('Target')
plt.title('Target over Time for Each num')
plt.legend()
plt.show()

In [ ]:
# 'datetime' 컬럼을 datetime 형식으로 변환
cluster_3_c['datetime'] = pd.to_datetime(cluster_3_c['datetime'])

# unique 'num' 값들
unique_nums = cluster_3_c['num'].unique()

# Plotting
plt.figure(figsize=(15, 10))

for num in unique_nums:
    subset = cluster_3_c[cluster_3_c['num'] == num]
    plt.plot(subset['datetime'], subset['target'], label=f'num {num}')

plt.xlabel('Datetime')
plt.ylabel('Target')
plt.title('Target over Time for Each num')
plt.legend()
plt.show()

In [ ]:
# 'datetime' 컬럼을 datetime 형식으로 변환
cluster_3_d['datetime'] = pd.to_datetime(cluster_3_d['datetime'])

# unique 'num' 값들
unique_nums = cluster_3_d['num'].unique()

# Plotting
plt.figure(figsize=(15, 10))

for num in unique_nums:
    subset = cluster_3_d[cluster_3_d['num'] == num]
    plt.plot(subset['datetime'], subset['target'], label=f'num {num}')

plt.xlabel('Datetime')
plt.ylabel('Target')
plt.title('Target over Time for Each num')
plt.legend()
plt.show()

### interpolate

In [ ]:
# 주피터에서 돌리기 어려우니 코랩에서 돌리는 것을 권장!
# 하단 주석 처리는 Pycaret을 위한 것입니다. Pycaret 사용 시 주석 해제 필요!

!pip install pycaret
import os

# 오직 중요한 로깅만 활성화하기 (선택적)
os.environ["PYCARET_CUSTOM_LOGGING_LEVEL"] = "CRITICAL"

# 설치된 라이브러리 확인
def what_is_installed():
    from pycaret import show_versions
    show_versions()

try:
    what_is_installed()
except ModuleNotFoundError:
    !pip install pycaret
    what_is_installed()

In [ ]:
# !pip install catboost
!pip install sktime

# data analysis and wrangling
import sys
import sktime
import random
import pandas as pd
import numpy as np
import matplotlib
import seaborn as sns
import sklearn as skl
import scipy.stats
import tqdm as tq
# import xgboost as xgb
import tensorflow as tf
import joblib
import missingno as msno

# EDA & visualization
import matplotlib.pyplot as plt
import statsmodels.api as sm           # 회귀 / 시계열 ... 일반적인 모델
import statsmodels.tsa.api as smt
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor # 다중공선성
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Time series
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.utils.plotting import plot_series

# ML Algorithms
import sklearn
from sklearn import set_config
from sklearn.utils import resample
from sklearn.linear_model import *
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score
from sklearn.decomposition import PCA

# model validation
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold, GroupKFold
from sklearn.model_selection import GridSearchCV, train_test_split, TimeSeriesSplit

# deep-learning libraries
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Reshape, GRU, RNN
tf.keras.backend.set_floatx('float64')

# 경고 무시
import warnings
warnings.filterwarnings('ignore')

SEED = 2
np.random.seed(SEED)

pd.set_option('display.max_columns', 30)

print("-------------------------- Python & library version --------------------------")
print(f"Python version: {sys.version}")
print(f"pandas version: {pd.__version__}")
print(f"numpy version: {np.__version__}")
print(f"matplotlib version: {matplotlib.__version__}")
print(f"seaborn version: {sns.__version__}")
print(f"joblib version: {joblib.__version__}")
print(f"missingno version: {msno.__version__}")
print(f"scipy version: {scipy.__version__}")
print(f"sktime version: {sktime.__version__}")
print(f"scikit-learn version: {skl.__version__}")
# print(f"lightgbm version: {lgb.__version__}")
# print(f"catboost version: {catboost.__version__}")
print(f"tqdm version: {tq.__version__}")
# print(f"xgboost version: {xgb.__version__}")
print(f"tensorflow version: {tf.__version__}")
print("------------------------------------------------------------------------------")


In [ ]:

train_df = pd.read_csv(r'C:\Users\USER\projects\data_study\train.csv', encoding='cp949')
test_df = pd.read_csv(r'C:\Users\USER\projects\data_study\test.csv', encoding='cp949')
sub = pd.read_csv(r'C:\Users\USER\projects\data_study\sample_submission.csv', encoding='cp949')

# renaming columns
train_df.columns = ['num','datetime','target','temperature','windspeed','humidity','precipitation','insolation','nelec_cool_flag','solar_flag']
test_df.columns = ['num','datetime','temperature','windspeed','humidity','precipitation','insolation','nelec_cool_flag','solar_flag']

# 데이터 형태 및 개수, 결측치 확인
train_df.info()

In [ ]:
def SMAPE(true, pred):
    return np.mean((np.abs(true-pred))/(np.abs(true) + np.abs(pred))) * 100

def datetimeTransform(data):
    data.insert(0, "date_time", pd.to_datetime(data["datetime"]))
    data.drop(columns=["datetime"], inplace=True)
    return data.head(25)

df_dict = {}
def separateDataFrame(data):
    for i in range(1, 61):
        df_dict[i] = data[data['num'] == i]
    return df_dict

def CDH(xs):
    ys = []
    for i in range(len(xs)):
        if i < 11:
            ys.append(np.sum(xs[:(i+1)]-26))
        else:
            ys.append(np.sum(xs[(i-11):(i+1)]-26))
    return np.array(ys)

def THI(data) :
    data['THI'] = round(1.8*data['temperature'] - 0.55*(1-(data['humidity']/100))*(1.8*data['temperature']-26) + 32, 2)
    # print(data['THI'].head())

    return data

In [ ]:
# train_df -> 건물번호 별 데이터프레임 정렬

# 냉방도시간 CDH(Cooling Degree Hour)
cdhs = np.array([])
for num in range(1,61,1):
    temp = train_df[train_df['num'] == num]
    cdh = CDH(temp['temperature'].values)
    cdhs = np.concatenate([cdhs, cdh])
train_df['CDH'] = cdhs

# 불쾌지수 함수
THI(train_df)
# 판다스 datetime 변환
datetimeTransform(train_df)
# 비전기냉방, 태양광 변수 삭제
# 60개 각각의 동일 건물별 모델링 시 비전기냉방과 태양광 여부는 필요없음 -> 삭제
train_df.drop(columns=['nelec_cool_flag','solar_flag'], inplace=True)
# 건물번호 구분 및 데이터 저장 함수
separateDataFrame(train_df)
df_dict[1]

In [ ]:
# 결측값 처리 일조
# discrete한 분포로 가장 최근에 기록된 일조값을 가져오는 형태로 결측치 보간
train_df['insolation'].value_counts()
test_df['insolation'] = test_df['insolation'].interpolate(method='pad')

# 결측치 채우기: 기온, 풍속, 습도, 강수량
# 기온, 풍속, 습도, 강수량은 'pad','linear','quadratic','cubic' 방법 중
# train 데이터에서 가장 성능이 뛰어난 보간법으로 결측치 보간 시행
# 우선 train 데이터 임의로 결측치를 생성
train_ = train_df.copy(deep=True)
def make_train_nan(col, n):
    new_list = []
    for idx, temp in enumerate(train_[col]):
        if idx%n==0:
            new_list.append(temp)
        else:
            new_list.append(np.nan)
    train_['{}'.format(col+'_nan')] = new_list
make_train_nan('temperature',3)
make_train_nan('windspeed',3)
make_train_nan('humidity',3)
make_train_nan('precipitation',6)
print(train_.iloc[:,-4:].isnull().sum())

# 결측치가 잘 생성됨을 확인하였으며 각 변수에 대해 4가지의 보간법을 시행한 후
# 가장 오차가 적은 보간법으로 결측치 보간
def compare_interpolate_methods(col, methods, metric):
    error_dict = dict()
    for method in methods:
        fillna = train_['{}'.format(col+'_nan')].interpolate(method=method)
        if fillna.isna().sum() != 0:
            fillna = fillna.interpolate(method='linear')
        error = metric(train_['{}'.format(col)], fillna)
        error_dict['{}'.format(method)] = error

    return error_dict
all_cols_error_dict = dict()
for col in ['temperature', 'windspeed', 'humidity', 'precipitation']:
    methods = ['pad','linear','quadratic','cubic']
    error_dict = compare_interpolate_methods(col, methods, mean_squared_error)
    all_cols_error_dict['{}'.format(col)] = error_dict

all_cols_error_df = pd.DataFrame(all_cols_error_dict)

fig, axes = plt.subplots(1,4, figsize = (18,5), sharey=False)
for i in range(len(all_cols_error_df.columns)):
    sns.lineplot(ax=axes[i], data=all_cols_error_df.iloc[:,i].transpose())

#기온 결측치 채우기
test_df['temperature'] = test_df['temperature'].interpolate(method='quadratic')
#마지막 na 채우기
test_df['temperature'] = test_df['temperature'].interpolate(method='linear')

#풍속 결측치 채우기
test_df['windspeed'] = test_df['windspeed'].interpolate(method='linear')

#습도 결측치 채우기
test_df['humidity'] = test_df['humidity'].interpolate(method='quadratic')
#마지막 na 채우기
test_df['humidity'] = test_df['humidity'].interpolate(method='linear')

#강수량 결측치 채우기
test_df['precipitation'] = test_df['precipitation'].interpolate(method='linear')

print(test_df.isna().sum())

In [ ]:
# test_df -> 건물번호 별 데이터프레임 정렬

# # 결측값 처리 pandas 내 선형보간 method 사용
# for i in range(60):
#     test_df.iloc[i*168:(i+1)*168, :]  = test_df.iloc[i*168:(i+1)*168, :].interpolate()

# 냉방도시간 CDH(Cooling Degree Hour)
cdhs = np.array([])
for num in range(1,61,1):
    temp = test_df[test_df['num'] == num]
    cdh = CDH(temp['temperature'].values)
    cdhs = np.concatenate([cdhs, cdh])
test_df['CDH'] = cdhs

# 불쾌지수 함수
THI(test_df)
# 판다스 datetime 변환
datetimeTransform(test_df)
# 비전기냉방, 태양광 변수 삭제
# 60개 각각의 동일 건물별 모델링 시 비전기냉방과 태양광 여부는 필요없음 -> 삭제
test_df.drop(columns=['nelec_cool_flag','solar_flag'], inplace=True)
# 건물번호 구분 및 데이터 저장 함수
test_df_dict = {}
for i in range(1, 61):
    test_df_dict[i] = test_df[test_df['num'] == i]
test_df_dict[1]

In [ ]:
# 건물 별 train_df test_df 의 건물번호를 삭제함
for i in range(1,61):
    df_dict[i].reset_index(drop=True, inplace=True)
    df_dict[i].drop(columns=['num'], inplace=True)
    test_df_dict[i].reset_index(drop=True, inplace=True)
    test_df_dict[i].drop(columns=['num'], inplace=True)

In [ ]:
def numPlot(data,i:int):
    ''' 건물번호로 분류한 시간에 따른 전력사용량 함수'''
    num_i = data[i].copy(deep=True)

    num_i.set_index('date_time',inplace=True)
    num_i.index = pd.to_datetime(num_i.index)
    return num_i.loc[:,'target']
numPlot(df_dict,1)

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pycaret.anomaly import *

# Assuming df_dict is already defined and contains your data
df_dict1 = {}
anomaly_counts = {}
interpolated_counts = {}

# Create subplots for anomaly detection results
fig_anomaly = make_subplots(rows=15, cols=4, subplot_titles=[f'Building {i}' for i in range(1, 61)])
# Create subplots for interpolated data results
fig_interpolated = make_subplots(rows=15, cols=4, subplot_titles=[f'Building {i}' for i in range(1, 61)])

for i in range(1, 61):
    data = df_dict[i].copy(deep=True)

    # set timestamp to index
    data.set_index('date_time', drop=True, inplace=True)

    # resample timeseries to hourly
    data = data.resample('H').sum()

    # create features from date
    data['day'] = [i.day for i in data.index]
    data['day_name'] = [i.day_name() for i in data.index]
    data['day_of_year'] = [i.dayofyear for i in data.index]
    data['week_of_year'] = [i.isocalendar()[1] for i in data.index]
    data['hour'] = [i.hour for i in data.index]
    data['is_weekday'] = [i.isoweekday() for i in data.index]

    # init setup
    s = setup(data, session_id=42,
              ordinal_features={'day_name': ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Sunday', 'Saturday']},
              numeric_features=['is_weekday'])

    # train model
    lof = create_model('lof')
    lof_results = assign_model(lof)

    # count anomalies
    anomaly_count = lof_results['Anomaly'].sum()
    anomaly_counts[i] = anomaly_count

    # filter out anomalies
    filtered_data = lof_results[lof_results['Anomaly'] != 1]

    # interpolate missing values
    interpolated_data = filtered_data.copy()
    interpolated_data['target'] = interpolated_data['target'].interpolate()

    # count interpolated values
    interpolated_count = interpolated_data['target'].isna().sum()
    interpolated_counts[i] = interpolated_count

    # Add anomaly detection plot to subplot
    outlier_dates = lof_results[lof_results['Anomaly'] == 1].index
    y_values = [lof_results.loc[j]['target'] for j in outlier_dates]
    row = (i-1) // 4 + 1
    col = (i-1) % 4 + 1
    fig_anomaly.add_trace(go.Scatter(x=lof_results.index, y=lof_results['target'], mode='lines', name=f'Building {i}'),
                          row=row, col=col)
    fig_anomaly.add_trace(go.Scatter(x=outlier_dates, y=y_values, mode='markers', name='Anomaly', marker=dict(color='red', size=5)),
                          row=row, col=col)

    # Add interpolated data plot to subplot
    fig_interpolated.add_trace(go.Scatter(x=interpolated_data.index, y=interpolated_data['target'], mode='lines', name=f'Building {i}'),
                               row=row, col=col)

    df_dict1[i] = lof_results
    df_dict1[i].drop(columns=['day', 'day_name', 'day_of_year', 'week_of_year', 'hour', 'is_weekday', 'Anomaly', 'Anomaly_Score'], inplace=True)
    df_dict1[i].reset_index(inplace=True)
    print(df_dict1[i])

# Update layout and show anomaly detection subplot
fig_anomaly.update_layout(height=3000, width=1200, title_text='전력사용량 - UNSUPERVISED ANOMALY DETECTION', showlegend=False)
fig_anomaly.show()

# Update layout and show interpolated data subplot
fig_interpolated.update_layout(height=3000, width=1200, title_text='전력사용량 - Interpolated Data (Outliers Removed)', showlegend=False)
fig_interpolated.show()

# Print anomaly and interpolation counts
for i in range(1, 61):
    print(f"Building {i}: {anomaly_counts[i]} anomalies, {interpolated_counts[i]} interpolations")


### 4조_분석코드(2차)